In [1]:
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Dataset, Datastore
from azureml.core import Dataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.core.runconfig import TensorflowConfiguration
import os

subscription_id = ### Fill the needed credential
resource_group =  ### Fill the needed credential
workspace_name = ### Fill the needed credential
  
ws = Workspace(subscription_id, resource_group, workspace_name)
  
datastore = Datastore.get(ws, "research")
dataset = Dataset.File.from_files(path=(datastore, 'Juuso files/ensamble_training_data')) # The location of the training data


experiment = Experiment(workspace=ws, name='re_train_ensemble_2')


dataset_input = DatasetConsumptionConfig('input',dataset).as_mount()

In [2]:
# Arguments for training

learning_rate = 0.003 #(at start)
epochs = 100
batch_size = 64
source_directory='./src' #directory where the script is found
script='train.py'

env_name= 'metamodel_training_env' 

# env_name= 'hus-env2' 
env = Environment.get(workspace = ws, name=env_name)



In [3]:
#run job

config = ScriptRunConfig(source_directory=source_directory,
                         script=script,
                        arguments = ['--dataset_input',dataset_input,
                                    '--learning_rate',learning_rate,
                                    '--batch_size',batch_size,
                                    '--epochs',epochs
                                    ],
                        compute_target='V100-low-priority',
                        environment = env,
                        )

run = experiment.submit(config)
run.display_name = ### Fill the name for the training run
